[View in Colaboratory](https://colab.research.google.com/github/dgketchum/IrrMapper/blob/master/DGK_TF_demo1.ipynb)

This is an Earth Engine <> TensorFlow demonstration notebook.  The default public runtime already has the tensorflow libraries we need installed.  The first step is to verify that by importing the TensorFlow library.  Run the code in the cell by clicking the run button on the left (hover on the `[ ]`).



In [0]:
#@title Import tensorflow library

import tensorflow as tf

Check the TensorFlow install by running a hello world operation (from the [Cloud ML example](https://cloud.google.com/ml-engine/docs/tensorflow/getting-started-training-prediction#run_a_simple_tensorflow_python_program)):

In [0]:
#@title Hello World TensorFlow

hello = tf.constant('Hello world!')
with tf.Session() as sess:
  print sess.run(hello)

Note that you can use "magic" commands by prepending an `!` to a bash command.  For example, here we will install a python library to enable us to connect to Google Drive.  Learn more about magic functions from **Code snippets** to the left.  The objective here is to enable access to thinhs in Drive that you may have exported from Earth Engine.

In [0]:
#@title Install the PyDrive library

# This only needs to be done once per notebook.
!pip install -U PyDrive

We need to import some authentication APIs so that we can read from Drive and/or a cloud storage bucket.  See the **Code snippets** to the left.

In [0]:
#@title Import authentication libraries

from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

**Authentication**.  The following will trigger the browser dance to authenticate.  Follow the link, copy the code from another browser window to the indicated field, then press return.  You should use the same account to authenticate here that you used to join the training group (which is hopefully the same account you use to login to Earth Engine, otherwise the exports will end up in the Drive of another account.)

In [0]:
#@title Authenticate

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

We've already generated some training data in Earth Engine.  Specifically, these are exported testing and training data from a very simple classification demo.  The script exports a training dataset, a testing dataset and the image data (in TFRecord format) on which to make predictions:

https://code.earthengine.google.com/a7ed957f3034825a54b6b546b8c5ce83

RUN THE EXPORTS

---

Note that the script exports to two places: Your drive account and a public cloud storage bucket.  You can grab the files you need from either place, as demonstrated in the following sections.  Here we'll use Drive, but note that code to use Cloud Storage is also here in case you need it.

In [0]:
#@title Load training/testing data from Earth Engine exports

# Specify the training file exported from EE.
# If you wish to use your own data, then
# replace the file ID, below, with your own file.
trainFileId = '1bLHhjGjKYXtdK_XAwC9636ZuxAKuGlmO' # nclinton version!
trainDownload = drive.CreateFile({'id': trainFileId})

# Create a local file of the specified name.
tfrTrainFile = 'training.tfrecord.gz'
trainDownload.GetContentFile(tfrTrainFile)
print 'Successfully downloaded training file?'
print tf.gfile.Exists(tfrTrainFile)

# Specify the test file.
# If you wish to use your own data, then
# replace the file ID, below, with your own file.
testFileId = '1PWakg7ygx-vRm5O_QKup6GIJup8LIvLy' # nclinton version!
testDownload = drive.CreateFile({'id': testFileId})

# Creates a local file of the specified name.
tfrTestFile = 'testing.tfrecord.gz'
testDownload.GetContentFile(tfrTestFile)
print 'Successfully downloaded testing file?'
print tf.gfile.Exists(tfrTestFile)

print 'Content of the working directory:'
!ls

Here we are going to read from the Drive file into a `tf.data.Dataset`.  ([Slide](https://docs.google.com/presentation/d/1fEf-oScgbC9zjbzI3K3jUlHf4JdmoSLFiG_H491FUmk/edit#slide=id.g3b76860e75_0_63)).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.

In [0]:
#@title Inspect the TFRecord dataset

driveDataset = tf.data.TFRecordDataset(tfrTrainFile, compression_type='GZIP')
iterator = driveDataset.make_one_shot_iterator()
foo = iterator.get_next()
with tf.Session() as sess:
    print sess.run([foo])

Define the structure of your data.  This includes the names of the bands you originally exported from Earth Engine and the name of the class property.  Unfortunately, these are called *features* in the TensorFlow context (not to be confused with an `ee.Feature`).  ([Slide](https://docs.google.com/presentation/d/1fEf-oScgbC9zjbzI3K3jUlHf4JdmoSLFiG_H491FUmk/edit#slide=id.g3b76860e75_0_67)).  Think of `columns` as a placeholder for the data that you're going to read in.

In [0]:
#@title Define the structure of the training/testing data

# Names of the features.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
label = 'landcover'
featureNames = list(bands)
featureNames.append(label)

# Feature columns
columns = [
  tf.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames
]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))
print featuresDict

Now we need to make a parsing function.  The parsing function reads data from a serialized example proto into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the feature for that example.  ([TF reference](https://www.tensorflow.org/programmers_guide/datasets#parsing_tfexample_protocol_buffer_messages), [Cloud ML reference](https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/cloudml-template/template/trainer/input.py#L61)).  

Here we make a parsing function for the TFRecord files we've been generating.  The check at the end is to print a single parsed example.

In [0]:
#@title Make and test a parsing function

def parse_tfrecord(example_proto):
  parsed_features = tf.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset
parsedDataset = driveDataset.map(parse_tfrecord, num_parallel_calls=5)

iterator = parsedDataset.make_one_shot_iterator()
foo = iterator.get_next()
with tf.Session() as sess:
    print sess.run([foo])

Another thing we might want to do as part of the input process is to create new features, for example NDVI.  Here are some helper functions for that.  Note that a and b are expected to be shape=[1] tensors and features is s dictionary of input tensors keyed by feature name.

In [0]:
#@title Make functions to add additional features

# Compute normalized difference of two inputs.  If denomenator is zero, add a small delta.
def normalizedDifference(a, b):
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.is_finite(nd), nd, nd_inf)

# Add normalized differences and 3-D coordinates to the dataset.  Shift the label to zero.
def addFeatures(features, label):
  features['NDVI'] = normalizedDifference(features['B5'], features['B4'])
  return features, label

Now we need to define an input function that will feed data from a file into a TensorFlow model.  Putting together what we've done so far, here is the complete function for input, parsing and feature engineering:

In [0]:
#@title Make an input function

def tfrecord_input_fn(fileName,
                      numEpochs=None,
                      shuffle=True,
                      batchSize=None):

  dataset = tf.data.TFRecordDataset(fileName, compression_type='GZIP')

  # Map the parsing function over the dataset
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)

  # Add additional features.
  dataset = dataset.map(addFeatures)

  # Shuffle, batch, and repeat.
  if shuffle:
    dataset = dataset.shuffle(buffer_size=batchSize * 10)
  dataset = dataset.batch(batchSize)
  dataset = dataset.repeat(numEpochs)

  # Make a one-shot iterator.
  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  return features, labels

The classifier we will use is a deep neural network (DNN) from the [`tf.estimator` package](https://www.tensorflow.org/api_docs/python/tf/estimator).  ([Slide](https://docs.google.com/presentation/d/1fEf-oScgbC9zjbzI3K3jUlHf4JdmoSLFiG_H491FUmk/edit#slide=id.g3b76860e75_0_59)).  First, define the input features, including the newly created NDVI column.  Here we specify an optimizer so that we can also set the learning rate.  Specify 7 nodes in the first hidden layer, 7 in the second and 5 in the third.  These are arbitrary demonstration numbers.  

Lastly, train the classifier.  In order to pass the classifier a single argument input function, use a lambda function to specify the number of epochs and batch size.  You could also specify the number of training steps here where steps = N / batchSize for a single epoch ([reference](https://developers.google.com/machine-learning/glossary/#epoch)).

In [0]:
#@title Make and train a classifier

inputColumns = {tf.feature_column.numeric_column(k) for k in ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI']}

learning_rate = 0.05
optimizer = tf.train.AdagradOptimizer(learning_rate)

classifier = tf.estimator.DNNClassifier(feature_columns=inputColumns,
                                  hidden_units=[5, 7, 5],
                                  n_classes=3,
                                  model_dir='output',
                                  optimizer=optimizer)

classifier.train(input_fn=lambda: tfrecord_input_fn(fileName=tfrTrainFile, numEpochs=8, batchSize=1))

Now that we have a trained classifier, we can evaluate it using the test set.  To do that, use the same input function on a different file.  Since this is the test set, just use one epoch and don't shuffle.  Here we just print the overall accuracy.

In [0]:
#@title Evaluate the classifier

accuracy_score = classifier.evaluate(
    input_fn=lambda: tfrecord_input_fn(fileName=tfrTestFile, numEpochs=1, batchSize=1, shuffle=False)
)['accuracy']

Training an estimator triggers storage of the state of the final model.  Unless you want subsequent runs to update previous model state, you may want to run the following (you will have to uncomment it first) to get rid of old model output.  Use with caution!

In [0]:
#@title Optionally delete model output

# !rm -rf output

**Optional**.  The following code cell checks that the classifier can work by training it and testing it on the files stored in the cloud storage bucket.  To see the code, toggle the form with the control to the right.

In [0]:
#@title Optional Cloud Storage way (No need to run)

inputColumns = {tf.feature_column.numeric_column(k) for k in ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI']}

learning_rate = 0.05
optimizer = tf.train.AdagradOptimizer(learning_rate)

classifier = tf.estimator.DNNClassifier(feature_columns=inputColumns,
                                  hidden_units=[5, 7, 5],
                                  n_classes=3,
                                  model_dir='output',
                                  optimizer=optimizer)

# TensorFlow can read directly from a cloud storage location, so all we need to do is specify the path.
tfrTrainFileCloud = 'gs://nclinton-training-temp/tf_demo_train_9a26cef21ab34f6257d0a250882124fcee_export.tfrecord.gz'
tfrTestFileCloud = 'gs://nclinton-training-temp/tf_demo_test_9a26cef21ab34f6257d0a250882124fcee_export.tfrecord.gz'

# Just check that you can see file(s):
print tf.gfile.Exists(tfrTrainFileCloud)
print tf.gfile.Exists(tfrTestFileCloud)

# Train and test, passing the cloud storage path into the input function. 
classifier.train(input_fn=lambda: tfrecord_input_fn(fileName=tfrTrainFileCloud, numEpochs=8, batchSize=1))
accuracy_score = classifier.evaluate(
    input_fn=lambda: tfrecord_input_fn(fileName=tfrTestFileCloud, numEpochs=1, batchSize=1, shuffle=False)
)['accuracy']

Get predictions on the evaluation dataset.  Note that we're going to make two iterators for this dataset.  The first one is just to see what's in there, to do a sanity check on the output.  We'll use the second one, below, to write the predictions to a TFRecord file.  

Note that you can get both the predicted class and support probabilities for that classification. 

In [0]:
#@title Make predictions on the test data

import itertools

# Do the prediction from the trained classifier.
checkPredictions = classifier.predict(
  input_fn=lambda: tfrecord_input_fn(fileName=tfrTestFile, numEpochs=1, batchSize=1, shuffle=False)
)

# Make a couple iterators.
iterator1, iterator2 = itertools.tee(checkPredictions, 2)

# Iterate over the predictions, printing the class_ids and posteriors.
for pred_dict in iterator1:
  class_id = pred_dict['class_ids']
  probability = pred_dict['probabilities']
  print class_id, probability

**Optional**.  To write into a TFRecord file, it helps to have alittle understanding of how the records are stored.   This next example is to practice building a single record and writing it.  Specifically, define a `tf.train.Example` [protocol buffer](https://developers.google.com/protocol-buffers/) and write it to a file.

In [0]:
#@title Demonstration of writing an Example

checkFilename = 'check.TFRecord'
writer = tf.python_io.TFRecordWriter(checkFilename)

example = tf.train.Example(
    features=tf.train.Features(
      feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=[1])),
          'posteriors': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=[0.1, 0.2, 0.3]))
      }
))

writer.write(example.SerializeToString())
writer.flush()
writer.close()

Now let's check that we can read our example back out of the file.

In [0]:
#@title Demonstration of reading an Example

checkDataset = tf.data.TFRecordDataset('check.TFRecord')

checkDict = {
    'prediction': tf.FixedLenFeature(shape=[1], dtype=tf.int64),
    'posteriors': tf.FixedLenFeature(shape=[3], dtype=tf.float32),
}

checkParsed = checkDataset.map(
    lambda example_proto: tf.parse_single_example(example_proto, checkDict))

iterator = checkParsed.make_one_shot_iterator()
foo = iterator.get_next()
with tf.Session() as sess:
    print sess.run([foo])

Now iterate over the predictions on the test data and try writing all those into a file.  For each prediction, we make a new `tf.Example` proto out of the prediction data, then write it.  Finally execute a shell command to see if we've successfully written the file.

See:

https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/core/example/feature.proto

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/how_tos/reading_data/convert_to_records.py

In [0]:
#@title Demonstration of writing predictions to a file

outputFilename = 'checkPredictions.TFRecord'
writer = tf.python_io.TFRecordWriter(outputFilename)
  
for pred_dict in iterator2:
  example = tf.train.Example(
    features=tf.train.Features(
      
      feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=pred_dict['class_ids'])),
          'probabilities': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=pred_dict['probabilities']))
      }
  ))
  writer.write(example.SerializeToString())
         
writer.close()
!ls -Al

Now it's time to classify the image from Earth Engine.  The way this happens is by exporting an image as a TFRecord file [announcement doc](https://docs.google.com/document/d/1njY_MKvXELEWvDaXmA56TFSteeiSytkziHBD9Pr8Q9I/edit?usp=sharing).  [The script for exporting the training and testing data](https://code.earthengine.google.com/a7ed957f3034825a54b6b546b8c5ce83) also exports a piece of the composite for classification.  Specifically, `Export.image` now accepts `'TFRecord'` for `fileFormat`. 

Theres some other new stuff in that export.  Specifically, note that we're exporting pixels in 256x256 patches for efficiency.  Also note that the image gets split into multiple TFRecord files in its destination folder.

Because there are multiple files that make up the image, use the Google PyDrive library to search for the files that match a particular prefix string.  Specifically, this is the name you specified in the JavaScript for the exported files.  Download all the Drive files that match that field, one of which is the JSON that we don't need as input to the model (but will need for import to Earth Engine after we've made predictions).  Lastly, print the list of filenames for a reality check.

See https://pythonhosted.org/PyDrive/filelist.html for pyDrive docs.  Here's where you can find the info on that query expression: https://developers.google.com/drive/api/v2/search-parameters#file_fields

In [0]:
#@title Find the exported image and JSON files in Drive

file_list = drive.ListFile({
    # You have to know this base filename from wherever you did the export.
    'q': 'title contains "tf_demo_image_9a26cef21ab34f6257d0a250882124fc"'
}).GetList()

fileNames = []
jsonFile = None
for gDriveFile in file_list:
  title = gDriveFile['title']
  # Download to the notebook server VM.
  gDriveFile.GetContentFile(title)
  # If the filename contains .gz, it's part of the image.
  if (title.find('gz') > 0):
    fileNames.append(gDriveFile['title'])
  if (title.find('json') > 0):
    jsonFile = title

# Make sure the files are in the right order.
fileNames.sort()

# Check the list of filenames to ensure there's nothing unintentional in there.
print(fileNames)


In [0]:
#@title Optional Cloud Storage way (No need to run)
# We'll need this for importing the classified image back to Earth Engine.
jsonFile = 'gs://nclinton-training-temp/tf_demo_image_9a26cef21ab34f6257d0a250882124fcmixer.json'

# TensorFlow can read directly from a cloud storage bucket.
# Ensure that the files are in order.
fileNames = [
    'gs://nclinton-training-temp/tf_demo_image_9a26cef21ab34f6257d0a250882124fc00000.tfrecord.gz',
    'gs://nclinton-training-temp/tf_demo_image_9a26cef21ab34f6257d0a250882124fc00001.tfrecord.gz'
]
print(fileNames)

We can feed this list of files directly to the Dataset constructor to make a combined dataset.  However, the the input function is slightly different from the previous ones.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.  Once the input function is defined that can handle the shape of the image data, all you need to do is feed it directly to the trained model to make predictions.

In [0]:
#@title Make an input function for exported image data

# You have to know the following from your export.
PATCH_WIDTH = 256
PATCH_HEIGHT = 256
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1]

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# Note that the tensors are in the shape of a patch, one patch for each band.
columns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) for k in bands
]

featuresDict = dict(zip(bands, columns))

# This function adds NDVI to a feature that doesn't have a label.
def addServerFeatures(features):
  return addFeatures(features, None)[0]
    
# This input function reads in the TFRecord files exported from an image.
# Note that because the pixels are arranged in patches, we need some additional
# code to reshape the tensors.
def predict_input_fn(fileNames):
  
  # Note that you can make one dataset from many files by specifying a list.
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')
  
  def parse_image(example_proto):
    parsed_features = tf.parse_single_example(example_proto, featuresDict)
    return parsed_features
  
  dataset = dataset.map(parse_image, num_parallel_calls=5)

  # Break our long tensors into many littler ones
  dataset = dataset.flat_map(lambda features: tf.data.Dataset.from_tensor_slices(features))
  
  # Add additional features (NDVI).
  dataset = dataset.map(addServerFeatures)
  
  # Read in batches corresponding to patch size.
  dataset = dataset.batch(PATCH_WIDTH * PATCH_HEIGHT)
  
  # Make a one-shot iterator.
  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next()

# Do the prediction from the trained classifier.
predictions = classifier.predict(
  input_fn=lambda: predict_input_fn(fileNames)
)


Name the TFRecord file you're going to create with a unique identifier for you (like your username).  We'll write this file directly into a temporary cloud storage bucket created for this training.  *The bucket will be deleted daily, so don't store anything in there*.

In [0]:
#@title Define output names

# INSERT YOUR USERNAME HERE (e.g. nclinton):
username = ''
baseName = 'gs://nclinton-training-temp/' + username
outputImageFile = baseName + '_predictions.TFRecord'
outputJsonFile = baseName + '_predictions.json'
print 'Writing to: ' + outputImageFile

We already have the predictions as a list.  Iterate over them as we did previously, except with some additional code to handle the shape.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  (Note: 5,620,989 pixels)

In [0]:
#@title Make predictions on the image data, write to a file

iter1, iter2 = itertools.tee(predictions, 2)

# Iterate over the predictions, printing the class_ids and posteriors.
# This is just to examine the first prediction.
for pred_dict in iter1:
  print pred_dict
  break # OK

# Instantiate the writer.
writer = tf.python_io.TFRecordWriter(outputImageFile)
  
# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since are predictions
# are already in the order of the exported data, our patches we create here
# will also be in the right order.
patch = [[], [], [], []]
curPatch = 1
for pred_dict in iter2:
  patch[0].append(pred_dict['class_ids'])
  patch[1].append(pred_dict['probabilities'][0])
  patch[2].append(pred_dict['probabilities'][1])
  patch[3].append(pred_dict['probabilities'][2])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + '...')
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
            
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    curPatch += 1 

writer.close()

At this stage, we should have a predictions TFRecord file sitting in the cloud storage bucket.  What you should look for is a file named `username_predictions.TFRecord` with non-zero size.  We also need to 

Note that you should also move the JSOn file downloaded earlier and give it the same base name as the TFRecord file with the predictions in it.  It's not necessary to do this, but will be helpful in the upload to Earth Engine command.



In [0]:
#@title Copy the JSON file to a cloud storage bucket

# Copy the JSON file so it has the same base name as the image.
!gsutil cp {jsonFile} {outputJsonFile}
!gsutil ls gs://nclinton-training-temp

Almost there!  Now we have a predictions image, sitting in a cloud storage bucket.  The purpose of doing it this way is to enable us to upload the image to Earth Engine from the cloud storage bucket.  This can be accomplished with the [Earth Engine command line tool](https://developers.google.com/earth-engine/command_line#upload).  But first we need to install the Earth Engine API and authenticate.

In [0]:
#@title Install the Earth Engine API

!pip install earthengine-api
!earthengine authenticate --quiet

Follow the link in the output above, copy the authorization link into the code cell below and run it to authenticate Earth Engine.

In [0]:
#@title Authentication for Earth Engine

!earthengine authenticate --authorization-code=<YOUR CODE HERE>

Let's just test the `earthengine` command by looking for help on the upload command.

In [0]:
#@title Get earthengine upload help

!earthengine upload image -h

Now we're ready to move the image file back to Earth Engine.  Note that we give both the image TFRecord file and the JSON file as arguments to `earthengine upload`.  Here's where it's useful to copy the JSON file to have a consistent basename with the image.

In [0]:
#@title Upload the classified image to Earth Engine

# Change the filenames to match your personal user folder in Earth Engine.
# e.g. users/nclinton/TF_nclinton_predictions
outputAssetID = '' 

!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {outputJsonFile}

In [0]:
#@title Check the status of the asset ingestion

import ee
ee.Initialize()

tasks = ee.batch.Task.list()
print tasks

Check the output in Earth Engine (nclinton version): https://code.earthengine.google.com/47ba19eedba20fad5d3df28fa2c4be1c  